In [39]:
import random


class Sentiment:


    NEGATIVE = "NEGATIVE"


    NEUTRAL = "NEUTRAL"

    POSITIVE = "POSITIVE"



class Review:


    def __init__(self, text, score):

        self.text = text


        self.score = score


    def getSentiment(self):

        if self.score <= 2:


            return Sentiment.NEGATIVE

        if self.score == 3:


            return Sentiment.NEUTRAL

        if self.score >= 4:


            return Sentiment.POSITIVE


class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.getSentiment() for x in self.reviews]

    def evenlyDistribute(self):
        negative = list(
            filter(lambda x: x.getSentiment() == Sentiment.NEGATIVE, self.reviews)
        )
        positive = list(
            filter(lambda x: x.getSentiment() == Sentiment.POSITIVE, self.reviews)
        )
        positive_shrunk = positive[: len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

#### Load Data


In [26]:
import json

filename = "data/Books_small_10000.json"
reviews = []
with open(filename, "r") as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review["reviewText"], review["overall"]))

print(reviews.__len__(), reviews[5].text, reviews[5].score, reviews[5].getSentiment())

10000 I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's. 5.0 POSITIVE


#### Data Preparation


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(reviews, test_size=0.33, random_state=42)
print(X_train.__len__(), X_test.__len__())

6700 3300


In [45]:
train_container = ReviewContainer(X_train)
test_container = ReviewContainer(X_test)
train_container.evenlyDistribute()
test_container.evenlyDistribute()

train_x = train_container.get_text()
test_x = test_container.get_text()

train_y = train_container.get_sentiment()
test_y = test_container.get_sentiment()

train_x[0], train_y[0], test_x[0], test_y[0]


("Out of the whole series this one was my favorite. I'm a little embarrassed to admit but at a point or two I did cry.. Lol. I guess these books aren't for everyone but I enjoyed reading them. The chemistry between Christian and Anastasia is one of a kind. Put all the crazy sex aside, there really is a good story going on.",
 'POSITIVE',
 "... Even down to the cigarettes with $ signs on them. Read the real thing. I know it's about 1000 pages, but it is well worth it",
 'NEGATIVE')

In [47]:
import pandas as pd

print(
    train_y.count(Sentiment.POSITIVE),
    train_y.count(Sentiment.NEGATIVE),
    test_y.count(Sentiment.POSITIVE),
    test_y.count(Sentiment.NEGATIVE),
)


436 436 208 208


#### Bag Of Words Vectorization


In [62]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()

In [63]:
train_x_vectors = vectorizer.fit_transform(train_x)

print(train_x[0])
print(train_x_vectors, train_x_vectors[0].toarray())

Out of the whole series this one was my favorite. I'm a little embarrassed to admit but at a point or two I did cry.. Lol. I guess these books aren't for everyone but I enjoyed reading them. The chemistry between Christian and Anastasia is one of a kind. Put all the crazy sex aside, there really is a good story going on.
  (0, 5512)	0.07981334659073278
  (0, 3423)	0.1264747685082817
  (0, 7533)	0.07641220458515317
  (0, 3432)	0.09247057197933146
  (0, 6411)	0.0970094707288688
  (0, 7951)	0.09446398844261626
  (0, 577)	0.2076077726757128
  (0, 7029)	0.1606391575728876
  (0, 1833)	0.1846821370329307
  (0, 347)	0.08843454980629596
  (0, 6293)	0.1312041760395943
  (0, 4472)	0.1606391575728876
  (0, 4264)	0.11872951816181422
  (0, 419)	0.25816654074276746
  (0, 423)	0.04612263050666229
  (0, 1433)	0.19282028097477016
  (0, 874)	0.14543485628398078
  (0, 1395)	0.19946962873387333
  (0, 7938)	0.11071467404937022
  (0, 6399)	0.10374680512167815
  (0, 2688)	0.12409023037996951
  (0, 2793)	0.159

### Classification


#### Linear SVM


In [50]:
from sklearn import svm

cls_svm = svm.SVC(kernel="linear")

In [64]:
cls_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [65]:
cls_svm.predict(vectorizer.transform(["very good book"]))

array(['POSITIVE'], dtype='<U8')

In [66]:
test_x_vectors = vectorizer.transform(test_x)
cls_svm.score(test_x_vectors, test_y)

0.8076923076923077

#### Naive Bayes


In [67]:
from sklearn.naive_bayes import GaussianNB

cls_gnb = GaussianNB()
cls_gnb.fit(train_x_vectors.toarray(), train_y)

cls_gnb.score(test_x_vectors.toarray(), test_y)

0.6610576923076923

#### Decision Tree


In [68]:
from sklearn.tree import DecisionTreeClassifier

cls_dec = DecisionTreeClassifier()
cls_dec.fit(train_x_vectors, train_y)

DecisionTreeClassifier()

In [69]:
cls_dec.predict(vectorizer.transform(["very good book"]))

array(['POSITIVE'], dtype='<U8')

In [70]:
cls_dec.score(test_x_vectors, test_y)

0.6274038461538461

#### Logistic Regression


In [71]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vectors, train_y)
lr.score(test_x_vectors, test_y)


0.8052884615384616

### Evaluation


In [72]:
from sklearn.metrics import f1_score

f1_score(
    test_y,
    cls_svm.predict(test_x_vectors),
    average=None,
    labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE],
)

array([0.80582524, 0.80952381])

In [73]:
test_set = ["very good book", "bad book do not buy", "horrible waste of time"]
new_test = vectorizer.transform(test_set)
cls_svm.predict(new_test)


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

### Tuning with Grid Search CV

In [75]:
from sklearn.model_selection import GridSearchCV

parameters = {"kernel": ("linear", "rbf"), "C": [1, 4, 8, 16, 32]}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ('linear', 'rbf')})

In [76]:
clf.score(test_x_vectors, test_y)

0.8197115384615384

### Saving Model